In [2]:
import pandas as pd

In [7]:
churn_data = pd.read_csv('Telco-Customer-Churn.csv')
churn_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [8]:
churn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [9]:
churn_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


#### Pipeline Creation for categorical columns & numerical columns

In [11]:
from sklearn.pipeline import make_pipeline

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [13]:
cat_pipeline = make_pipeline(SimpleImputer(strategy='constant', fill_value='missing'),
                            OneHotEncoder())

In [14]:
cat_data_tf = cat_pipeline.fit_transform(churn_data[cat_cols]).toarray()

NameError: name 'cat_cols' is not defined

In [ ]:
num_pipeline = make_pipeline(SimpleImputer(strategy='median'),
                             MinMaxScaler())                            

In [ ]:
num_data_tf = num_pipeline.fit_transform(churn_data[num_cols])

In [ ]:
import numpy as np

In [39]:
feature_data = np.hstack([cat_data_tf, num_data_tf, churn_data.luxury_car_user.reshape(-1,1)])

C:\Users\awant\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [49]:
feature_data.shape

(50000, 14)

In [40]:
target_data = churn_data.churn

In [45]:
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [43]:
final_pipeline = make_pipeline(SelectKBest(k=8), RandomForestClassifier(n_estimators=20))

In [46]:
trainX, testX, trainY, testY = train_test_split(feature_data,target_data)

In [47]:
final_pipeline.fit(trainX,trainY)

Pipeline(memory=None,
     steps=[('selectkbest', SelectKBest(k=8, score_func=<function f_classif at 0x000001F05973DB70>)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decre...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [48]:
final_pipeline.score(testX,testY)

0.66072

In [50]:
from sklearn.model_selection import GridSearchCV

In [51]:
params = {
    'selectkbest__k':[5,8,12],
    'randomforestclassifier__n_estimators':[10,30,50]
}

In [52]:
grid_model = GridSearchCV(final_pipeline,param_grid=params, cv=5, n_jobs=-1)

In [53]:
grid_model.fit(trainX,trainY)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('selectkbest', SelectKBest(k=8, score_func=<function f_classif at 0x000001F05973DB70>)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decre...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'selectkbest__k': [5, 8, 12], 'randomforestclassifier__n_estimators': [10, 30, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [54]:
grid_model.best_params_

{'randomforestclassifier__n_estimators': 50, 'selectkbest__k': 12}

In [55]:
grid_model.best_score_

0.7619733333333333

In [56]:
grid_model.score(testX,testY)

0.7592

In [57]:
from sklearn.compose import ColumnTransformer

In [74]:
num_cols

['avg_dist',
 'avg_rating_by_driver',
 'avg_rating_of_driver',
 'avg_surge',
 'surge_pct',
 'weekday_pct',
 'trips_in_first_30_days']

#### Using column transformer

In [95]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_cols),
        ('cat', cat_pipeline, cat_cols)])

In [96]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('preprocessor',preprocessor),
                           ('selectkbest', SelectKBest(k=8)),
                           ('classifier',RandomForestClassifier(n_estimators=20))])

In [97]:
#pipeline = make_pipeline(preprocessor, SelectKBest(k=8), RandomForestClassifier(n_estimators=20))

In [98]:
num_pipeline

Pipeline(memory=None,
     steps=[('simpleimputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbose=0)), ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1)))])

In [99]:
params = {
    'preprocessor__num__simpleimputer__strategy':['mean','median'],
    'selectkbest__k':[12,13],
    'classifier__n_estimators':[50,70]
}

In [100]:
grid_model = GridSearchCV(pipeline, param_grid=params,cv=5, n_jobs=-1)

In [101]:
feature_data = churn_data.drop('churn',axis=1)
target_data = churn_data.churn
trainX, testX, trainY, testY = train_test_split(feature_data,target_data)

In [102]:
grid_model.fit(trainX,trainY)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', Pipeline(memory=None,
     steps=[('simpleimputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median',...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'preprocessor__num__simpleimputer__strategy': ['mean', 'median'], 'selectkbest__k': [12, 13], 'classifier__n_estimators': [50, 70]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [103]:
grid_model.best_score_

0.7528

In [104]:
grid_model.best_params_

{'classifier__n_estimators': 70,
 'preprocessor__num__simpleimputer__strategy': 'mean',
 'selectkbest__k': 12}